# DFL benchmark - training

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
# from IPython.display import Video
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Configuration

In [2]:
DEBUG = False
ALLDATA_USE = False

In [3]:
class CFG:    
    IMAGE_SIZE = 256
    NUM_IMAGES = 8
    INPUT_CHANNEL = 3
    
    # SAVE_IMAGE_INTERVAL = 0.32

## train valid data setting

In [4]:
err_tol = {
    'challenge': [ 0.30, 0.40, 0.50, 0.60, 0.70 ],
    'play': [ 0.15, 0.20, 0.25, 0.30, 0.35 ],
    'throwin': [ 0.15, 0.20, 0.25, 0.30, 0.35 ]
}

validation_video = ['cfbe2e94_0','cfbe2e94_1']
    
#                  '35bd9041_0','35bd9041_1',
#                  '407c5a9e_1','4ffd5986_0','cfbe2e94_0','cfbe2e94_1',
#                  '9a97dae4_1','ecf251d4_0',]
    
    
if ALLDATA_USE:
    video_id_split = {
        'val':['3c993bd2_0','3c993bd2_1'],
        'train':['1606b0e6_0','1606b0e6_1','35bd9041_0','35bd9041_1',
                 '407c5a9e_1','4ffd5986_0','cfbe2e94_0','cfbe2e94_1',
                 '9a97dae4_1','ecf251d4_0']
    }

event_names = ['challenge', 'throwin', 'play']

# Load Data

In [5]:
df = pd.read_csv("/workdir/work/input/train.csv")
df = df[["video_id", "time", "event"]]
display(df)

,video_id,time,event
0,1606b0e6_0,200.265822,start
1,1606b0e6_0,201.150000,challenge
2,1606b0e6_0,202.765822,end
3,1606b0e6_0,210.124111,start
4,1606b0e6_0,210.870000,challenge
...,...,...,...
11213,ecf251d4_0,3056.587000,challenge
11214,ecf251d4_0,3058.072895,end
11215,ecf251d4_0,3068.280519,start
11216,ecf251d4_0,3069.547000,throwin


In [6]:
df_check = df[df["video_id"].isin(validation_video)]
display(df_check)

,video_id,time,event
8652,cfbe2e94_0,229.321518,start
8653,cfbe2e94_0,230.200000,play
8654,cfbe2e94_0,232.520000,play
8655,cfbe2e94_0,234.016200,end
8656,cfbe2e94_0,246.666301,start
...,...,...,...
10233,cfbe2e94_1,3562.660000,play
10234,cfbe2e94_1,3563.835896,end
10235,cfbe2e94_1,3572.500727,start
10236,cfbe2e94_1,3574.340000,throwin


In [7]:
df_check[df_check["video_id"] == validation_video[0]]

,video_id,time,event
8652,cfbe2e94_0,229.321518,start
8653,cfbe2e94_0,230.200000,play
8654,cfbe2e94_0,232.520000,play
8655,cfbe2e94_0,234.016200,end
8656,cfbe2e94_0,246.666301,start
...,...,...,...
9470,cfbe2e94_0,3208.440000,play
9471,cfbe2e94_0,3209.571684,end
9472,cfbe2e94_0,3214.407029,start
9473,cfbe2e94_0,3215.640000,throwin


In [8]:
result_dir = f"/workdir/work/output/3dcnn_validdata_allframe"
if not os.path.exists(result_dir):
    os.makedirs(result_dir, exist_ok=True)

In [9]:
def extract_validation_images(video_id):
    saved_frame_list = []
    img_saved_list = []
    video_path = f"/workdir/work/input/train/{video_id}.mp4"
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        TODO
    fps = cap.get(cv2.CAP_PROP_FPS)
    time_interval = 1/fps

    df_video = df[df.video_id == video_id]

    #crr_statu => background, play, challenge, throwin
    time_and_event = df_video[['time','event']].values
    print(f"{len(time_and_event)} events")
    event_idx = 0
    frame_num = 0
    while True:
        if DEBUG and frame_num > 100:
            break
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        file_name_list = ""
        for i in range(CFG.NUM_IMAGES):
            successed, img = cap.read()
            if not successed:
                break
            save_frame = frame_num + i
            out_file = f'{result_dir}/{video_id}_{save_frame:06}.jpg'
            if not save_frame in img_saved_list:
                cv2.imwrite(out_file, img)
            img_saved_list.append(save_frame)
            file_name = f"{video_id}_{save_frame:06}.jpg"

            if len(file_name_list) == 0:
                file_name_list = out_file
            else:
                file_name_list += "," + out_file
#             file_name_list.append([file_name])

        if i == CFG.NUM_IMAGES - 1:
            saved_frame_list.append([video_id, frame_num, file_name_list])
        if not successed:
            break
        
        # frame_num += int(fps*CFG.SAVE_IMAGE_INTERVAL)
        frame_num += 1    
    return saved_frame_list

In [10]:
saved_df = pd.DataFrame()

for video_id in validation_video:
    print(video_id)     
    saved_list = extract_validation_images(video_id)
    if len(saved_df) > 0:
        tmp = pd.DataFrame(saved_list, columns=["video_id", "frame", "file_name"])
        saved_df = pd.concat([saved_df, tmp])
    else:
        saved_df = pd.DataFrame(saved_list, columns=["video_id", "frame", "file_name"])
display(saved_df)
saved_df.to_csv("/workdir/work/output/3dcnn_valid_images_allframe.csv", index=False)
print('done')

cfbe2e94_0
823 events
cfbe2e94_1
763 events


,video_id,frame,file_name
0,cfbe2e94_0,0,/workdir/work/output/3dcnn_validdata_allframe/...
1,cfbe2e94_0,1,/workdir/work/output/3dcnn_validdata_allframe/...
2,cfbe2e94_0,2,/workdir/work/output/3dcnn_validdata_allframe/...
3,cfbe2e94_0,3,/workdir/work/output/3dcnn_validdata_allframe/...
4,cfbe2e94_0,4,/workdir/work/output/3dcnn_validdata_allframe/...
...,...,...,...
89914,cfbe2e94_1,89914,/workdir/work/output/3dcnn_validdata_allframe/...
89915,cfbe2e94_1,89915,/workdir/work/output/3dcnn_validdata_allframe/...
89916,cfbe2e94_1,89916,/workdir/work/output/3dcnn_validdata_allframe/...
89917,cfbe2e94_1,89917,/workdir/work/output/3dcnn_validdata_allframe/...


done


In [11]:
saved_df["file_name"].iloc[1]

'/workdir/work/output/3dcnn_validdata_allframe/cfbe2e94_0_000001.jpg,/workdir/work/output/3dcnn_validdata_allframe/cfbe2e94_0_000002.jpg,/workdir/work/output/3dcnn_validdata_allframe/cfbe2e94_0_000003.jpg,/workdir/work/output/3dcnn_validdata_allframe/cfbe2e94_0_000004.jpg,/workdir/work/output/3dcnn_validdata_allframe/cfbe2e94_0_000005.jpg,/workdir/work/output/3dcnn_validdata_allframe/cfbe2e94_0_000006.jpg,/workdir/work/output/3dcnn_validdata_allframe/cfbe2e94_0_000007.jpg,/workdir/work/output/3dcnn_validdata_allframe/cfbe2e94_0_000008.jpg'

In [12]:
len(saved_df["file_name"].iloc[1].split(","))

8